In [1]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

# Initialize wandb API
api = wandb.Api()

# Replace 'your_sweep_id' with your actual sweep ID
sweep_id = 'maximes_crew/S3_SC_runs/qra6dns9'

# Fetch the sweep object
sweep = api.sweep(sweep_id)

# Fetch all runs in the sweep
runs = sweep.runs

sweep_params = ['lr', 'pdrop', 'scheduler_patience', 'scheduler_factor', 'dt_max']

In [3]:
runs

<Runs maximes_crew/S3_SC_runs>

In [16]:
# Initialize an empty DataFrame to hold the relevant data
data = []

# Extract the necessary data from each run
for run in runs:
    config = run.config
    summary = run.summary
    history = run.history()
    n_layers = config.get('nb_layers')
    n_hiddens = config.get('nb_hiddens')
    lr = config.get('lr')
    dt_min = config.get('dt_min')
    dt_max = config.get('dt_max')
    pdrop = config.get('pdrop')
    sched_patience = config.get('scheduler_patience')
    sched_factor = config.get('scheduler_factor')
    test_acc = history['test acc'].dropna().iloc[-1] if 'test acc' in history.columns else None


    if test_acc is not None:
        data.append({
            'n_layers': n_layers,
            'n_hiddens': n_hiddens,
            'lr': lr,
            'dt_min': dt_min,
            'dt_max': dt_max,
            'pdrop' : pdrop,
            'scheduler_patience' : sched_patience,
            'scheduler_factor' : sched_factor,
            'test_acc': test_acc
        })



In [17]:
data

[{'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 1.5,
  'pdrop': 0.25,
  'scheduler_patience': 10,
  'scheduler_factor': 0.9,
  'test_acc': 0.9286620639534884},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 1.5,
  'pdrop': 0.25,
  'scheduler_patience': 5,
  'scheduler_factor': 0.9,
  'test_acc': 0.9317463662790698},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 1.5,
  'pdrop': 0.25,
  'scheduler_patience': 5,
  'scheduler_factor': 0.9,
  'test_acc': 0.928482558139535},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 1.5,
  'pdrop': 0.25,
  'scheduler_patience': 5,
  'scheduler_factor': 0.9,
  'test_acc': 0.9339375},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.01,
  'dt_min': 0.01,
  'dt_max': 1.5,
  'pdrop': 0.25,
  'scheduler_patience': 10,
  'scheduler_factor': 0.7,
  'test_acc': 0.9253895348837208},
 {'n_layers': 3,
  'n_hiddens': 512,
  'lr': 0.0

In [19]:
# Convert the data to a DataFrame for easier manipulation
df = pd.DataFrame(data)

# Group by n_layers and lr, then compute the mean of best_valid_acc for each group
df_grouped = df.groupby(sweep_params).agg({'test_acc': 'mean'}).reset_index()

# Calculate the number of runs for each line configuration
df['run_count'] = df.groupby(sweep_params).transform('size')

# Add the run count to the grouped dataframe
df_grouped['run_count'] = df_grouped[sweep_params].apply(lambda x: df[(df[sweep_params] == x).all(axis=1)]['run_count'].iloc[0], axis=1)

df_grouped_sorted = df_grouped.sort_values(by='test_acc', ascending=False).reset_index(drop=True)

In [20]:
df_grouped_sorted[0:10]

,lr,pdrop,scheduler_patience,scheduler_factor,dt_max,test_acc,run_count
0,0.005,0.50,5,0.7,1.0,0.940627,3
1,0.005,0.50,5,0.7,0.5,0.940111,3
2,0.005,0.50,5,0.9,1.0,0.939720,3
3,0.010,0.50,5,0.7,1.0,0.939534,3
4,0.005,0.50,10,0.7,1.0,0.939358,3
5,0.010,0.50,5,0.7,0.5,0.939116,3
6,0.005,0.50,10,0.7,0.5,0.937844,3
7,0.001,0.50,5,0.9,1.0,0.937145,3
8,0.001,0.50,5,0.7,0.5,0.937056,3
9,0.010,0.25,5,0.7,0.5,0.937049,3
